In [34]:
# imports
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import time
import seaborn
import os
import string
from PIL import Image

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [56]:
# Create a mapping from the image files to the character they represents
character_list = list(string.digits + string.ascii_uppercase + string.ascii_lowercase)
nums = range(1,len(a)+1)
file_to_character = dict(zip(nums, character_list))

In [132]:
# Take the median size of the images as the standard_size for 
# normalization of the images
path = '/home/reza/MachineLearning/Hand-Writing-Recognition/data/English/img/'
img_sizes = []
for folder in os.listdir(path):
    for img_file in os.listdir(path+'/'+folder+'/'):
        img = Image.open(path+'/'+folder+'/'+img_file)
        img_sizes.append(img.size)

# resize function only takes int values
STANDARD_SIZE = map(int, median(img_sizes, axis=0))

In [399]:
# Create an image class that allows us to represent the image in matrix or vector from
class img():
    def __init__(self, img_file=None, img_path=None):
        if img_file is None or file_name is None:
            raise Exception('Must give an image file as input along'
                            'with its file name')
        self.img_matrix = None
        self.img_vector = None
        self.img_label = None
        self.img_file = img_file
        self.img_path = img_path
        
    def img_to_matrix(self):
        """
        Extract a matrix representation of the image
        """
        # Resize the image to a standard size
        img_resized = self.img_file.resize(STANDARD_SIZE)
        self.img_matrix = array(img_resized)
#         self.img_matrix = list(img_resized.getdata())
#         if type(self.img_matrix[0]) is tuple:
#             self.img_matrix = map(list, self.img_matrix)
#             self.img_matrix = np.array(self.img_matrix)
#         else:
#         self.img_matrix = np.array(self.img_matrix)
#         self.img_matrix.shape = (1, self.img_matrix.shape[0]) 
    def flatten_image(self):
        """
        Flatten the matrix representation of the image. If the
        matrix has shape (m, n) then the vector representation
        has shape (1, m * n)
        """
        if self.img_matrix is None:
            self.img_to_matrix()
        s = self.img_matrix.shape[0] * self.img_matrix.shape[1]
        self.img_vector = self.img_matrix.reshape(1, s)[0]

    def get_img_label(self):
        """"
        Sets the label for the image file, i.e, the English
        character the image is showing
        """
        file_name = self.img_path.split('/')[-1][3:6]
        self.img_label = file_to_character[int(file_name)]
        
img1 = img(img_file=img_file, img_path='blahblah/im-001')
img1.img_label
img1.img_to_matrix()
print img1.img_matrix
img1.flatten_image()
img1.img_vector
# img1.get_img_label()
# img1.img_label


[[158 158 157 ..., 153 153 152]
 [159 159 157 ..., 154 154 154]
 [157 159 159 ..., 155 154 155]
 ..., 
 [162 158 155 ..., 154 154 153]
 [152 154 155 ..., 154 154 152]
 [154 157 158 ..., 154 154 151]]


array([158, 158, 157, ..., 154, 154, 151], dtype=uint8)

In [400]:
# Gather features of the image for building the training data
path = '/home/reza/MachineLearning/Hand-Writing-Recognition/data/English/img/'
data = []
labels = []
for folder in os.listdir(path):
    for img_file in os.listdir(path+'/'+folder+'/'):
        img_file = Image.open(path+'/'+folder+'/'+img_file)
        img_path = img_file.filename
        img_file = img_file.convert('L')
        
        # Open in the image file
        curr_img = img(img_file=img_file, img_path=img_path)
        
        # Take a vectore representation of the image
        curr_img.flatten_image()
        data.append(curr_img.img_vector)
        
        # add the label
        curr_img.get_img_label()
        labels.append(curr_img.img_label)

data = np.array(data)

In [ ]:
from sklearn.decomposition import RandomizedPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score

# Create the training data
pca = RandomizedPCA(n_components=200)
X = pca.fit_transform(data)
df = DataFrame(X)
response = 'label'
df[response] = labels

# Divide data to test and train
mask = np.random.rand(len(df)) < 0.75
train_dat = df[mask]
test_dat = df[~mask]
y_test = np.array(test_dat[response])
test_dat.drop(response, axis=1, inplace=True)

# Run random forest on the train model and pick the number
# of trees used for the model
rf = RandomForestClassifier(n_estimators=100, oob_score=True)
rf.fit(train_dat.drop(response, axis=1), train_dat[response])
prediction = rf.predict(test_dat)

# # Check error percentage on the test set and tune parameters
mean(prediction != y_test)

In [496]:
# Lets test this on a sample image
path = '/home/reza/Downloads/test.jpg'
img_file = Image.open(path)
img_path = img_file.filename
img_file = img_file.convert('L')
        
# Open in the image file
curr_img = img(img_file=img_file, img_path=img_path)
        
# Take a vectore representation of the image
curr_img.flatten_image()
test_point = pca.transform(curr_img.img_vector)

# Predict the label for the test point
rf.predict(test_point)

array(['E'], dtype=object)

In [493]:
# pca.fit_transform(data)
pca.transform(curr_img.img_vector)

array([[  6.34577558e+03,   6.84556293e+02,   5.59208035e+01,
         -8.46845980e+02,  -4.41662885e+02,  -2.76884066e+01,
          1.44019059e+02,   3.21069032e+02,   4.93002186e+02,
         -1.74616988e+02,  -1.44020397e+02,  -2.81885666e+02,
         -4.09568842e+02,   1.35128162e+02,  -2.57477347e+02,
          4.05459057e+01,   3.06752019e+02,  -2.25583296e+02,
         -3.40521745e+01,   5.80173919e+01,  -1.31436216e+02,
          5.92273735e+00,  -1.88621552e+01,  -4.34929528e+02,
          9.64304280e+01,   4.16957982e+01,   1.63363781e+02,
         -7.13771988e+01,  -2.06998188e+02,   1.45761422e+02,
          1.26428248e+02,   7.63334545e+01,  -1.76459278e+02,
         -1.05714390e+02,   1.10165275e+02,  -1.46324568e+02,
          9.25733320e+01,   8.84218509e+01,  -1.71506714e+02,
          2.86410992e+01,   8.56800328e+01,  -7.13905291e+01,
          3.15125133e+01,   8.33700841e+01,  -9.23399038e+01,
         -5.19382454e+01,  -1.97859510e+02,   1.43309754e+02,
        

In [494]:
pca.transform(curr_img.img_vector)

array([[  6.34577558e+03,   6.84556293e+02,   5.59208035e+01,
         -8.46845980e+02,  -4.41662885e+02,  -2.76884066e+01,
          1.44019059e+02,   3.21069032e+02,   4.93002186e+02,
         -1.74616988e+02,  -1.44020397e+02,  -2.81885666e+02,
         -4.09568842e+02,   1.35128162e+02,  -2.57477347e+02,
          4.05459057e+01,   3.06752019e+02,  -2.25583296e+02,
         -3.40521745e+01,   5.80173919e+01,  -1.31436216e+02,
          5.92273735e+00,  -1.88621552e+01,  -4.34929528e+02,
          9.64304280e+01,   4.16957982e+01,   1.63363781e+02,
         -7.13771988e+01,  -2.06998188e+02,   1.45761422e+02,
          1.26428248e+02,   7.63334545e+01,  -1.76459278e+02,
         -1.05714390e+02,   1.10165275e+02,  -1.46324568e+02,
          9.25733320e+01,   8.84218509e+01,  -1.71506714e+02,
          2.86410992e+01,   8.56800328e+01,  -7.13905291e+01,
          3.15125133e+01,   8.33700841e+01,  -9.23399038e+01,
         -5.19382454e+01,  -1.97859510e+02,   1.43309754e+02,
        